In [16]:
# VGG16 (Train-Test)

#coding=utf-8
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

start = time.clock() #计算开始时间

mnist = input_data.read_data_sets("/MNIST_data", one_hot=True)

# 定义网络超参数
learning_rate = 0.001
training_iters = 100000
batch_size = 64
display_step = 20

# 定义网络参数
n_input = 784 # 输入的维度
n_classes = 10 # 标签的维度
dropout = 0.8 # Dropout 的概率

# 占位符输入
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# 卷积操作
def conv2D(name, l_input, w, b):
    return tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(l_input, w, strides=[1, 1, 1, 1], padding='SAME'),b), name=name)

# 最大下采样操作
def maxPool2D(name, l_input, k):
    return tf.nn.max_pool(l_input, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME', name=name)

# 归一化操作
def norm(name, l_input, lsize=4):
    return tf.nn.lrn(l_input, lsize, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name=name)

def convLevel(i,input,type):
    num = i
    out = conv2D('conv'+str(num),input,weights['wc'+str(num)],biases['bc'+str(num)])
    if  type=='p':
        out = maxPool2D('pool'+str(num),out, k=2)
        out = norm('norm'+str(num),out, lsize=4)
    return out

def VGG(x,weights,biases,dropout):
    x = tf.reshape(x,shape=[-1,28,28,1])

    input = x

    for i in range(16):
        i += 1
        if(i==2) or (i==4) or (i==12) : # 根据模型定义还需要更多的POOL化，但mnist图片大小不允许。
            input = convLevel(i,input,'p')
        else:
            input = convLevel(i,input,'c')
            
    # 全连接层，先把特征图转为向量
    input = tf.reshape(input, [-1, weights['wd1'].get_shape().as_list()[0]]) 
    input = tf.nn.relu(tf.matmul(input, weights['wd1']) + biases['bd1'], name='fc1') 
    # 全连接层
    input = tf.nn.relu(tf.matmul(input, weights['wd2']) + biases['bd2'], name='fc2') # Relu activation

    # 网络输出层
    out = tf.matmul(input, weights['out']) + biases['out']
    return out
            
weights ={
    'wc1':tf.Variable(tf.random_normal([3,3,1,64])),
    'wc2':tf.Variable(tf.random_normal([3,3,64,64])),
    'wc3':tf.Variable(tf.random_normal([3,3,64,128])),
    'wc4':tf.Variable(tf.random_normal([3,3,128,128])),

    'wc5':tf.Variable(tf.random_normal([3,3,128,256])),
    'wc6':tf.Variable(tf.random_normal([3,3,256,256])),
    'wc7':tf.Variable(tf.random_normal([3,3,256,256])),
    'wc8':tf.Variable(tf.random_normal([3,3,256,256])),
    
    'wc9':tf.Variable(tf.random_normal([3,3,256,512])),
    'wc10':tf.Variable(tf.random_normal([3,3,512,512])),
    'wc11':tf.Variable(tf.random_normal([3,3,512,512])),
    'wc12':tf.Variable(tf.random_normal([3,3,512,512])),
    'wc13':tf.Variable(tf.random_normal([3,3,512,512])),
    'wc14':tf.Variable(tf.random_normal([3,3,512,512])),
    'wc15':tf.Variable(tf.random_normal([3,3,512,512])),
    'wc16':tf.Variable(tf.random_normal([3,3,512,256])),

    'wd1':tf.Variable(tf.random_normal([4096,4096])),
    'wd2':tf.Variable(tf.random_normal([4096,4096])),
    'out':tf.Variable(tf.random_normal([4096,n_classes])),
}

biases ={
    'bc1':tf.Variable(tf.zeros([64])),
    'bc2':tf.Variable(tf.zeros([64])),
    'bc3':tf.Variable(tf.zeros([128])),
    'bc4':tf.Variable(tf.zeros([128])),
    'bc5':tf.Variable(tf.zeros([256])),
    'bc6':tf.Variable(tf.zeros([256])),
    'bc7':tf.Variable(tf.zeros([256])),
    'bc8':tf.Variable(tf.zeros([256])),
    'bc9':tf.Variable(tf.zeros([512])),
    'bc10':tf.Variable(tf.zeros([512])),
    'bc11':tf.Variable(tf.zeros([512])),
    'bc12':tf.Variable(tf.zeros([512])),
    'bc13':tf.Variable(tf.zeros([512])),
    'bc14':tf.Variable(tf.zeros([512])),
    'bc15':tf.Variable(tf.zeros([512])),
    'bc16':tf.Variable(tf.zeros([256])),

    'bd1':tf.Variable(tf.zeros([4096])),
    'bd2':tf.Variable(tf.zeros([4096])),
    'out':tf.Variable(tf.zeros([n_classes])),
}

pred = VGG(x, weights, biases, keep_prob)

# 定义损失函数和学习步骤
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# 测试网络
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

config = tf.ConfigProto(
        device_count = {'GPU': 0}
    )

sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

saver = tf.train.Saver()

# 开启一个训练
step = 1
# Keep training until reach max iterations
while step * batch_size < training_iters:
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    # 获取批数据
    sess.run(optimizer, feed_dict={x: batch_xs, y: batch_ys, keep_prob: dropout})
    if step % display_step == 0:
        # 计算精度
        acc = sess.run(accuracy, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
        # 计算损失值
        loss = sess.run(cost, feed_dict={x: batch_xs, y: batch_ys, keep_prob: 1.})
        print ("Iter " + str(step*batch_size) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy = " + "{:.5f}".format(acc))
    step += 1
print ("Optimization Finished!")

# 计算测试精度
print ("Testing Accuracy:", sess.run(accuracy, feed_dict={x: mnist.test.images[:256], y: mnist.test.labels[:256], keep_prob: 1.}))

saver.save(sess, './model/mymodel', step*batch_size)

end = time.clock() #计算程序结束时间
print(("running time is %g s") % (end-start))

sess.close()

Extracting /MNIST_data\train-images-idx3-ubyte.gz
Extracting /MNIST_data\train-labels-idx1-ubyte.gz
Extracting /MNIST_data\t10k-images-idx3-ubyte.gz
Extracting /MNIST_data\t10k-labels-idx1-ubyte.gz
Iter 1280, Minibatch Loss= 11252216.000000, Training Accuracy = 0.17188
Iter 2560, Minibatch Loss= 3721868.500000, Training Accuracy = 0.35938
Iter 3840, Minibatch Loss= 740774.625000, Training Accuracy = 0.51562
Iter 5120, Minibatch Loss= 580353.062500, Training Accuracy = 0.60938
Iter 6400, Minibatch Loss= 1039277.312500, Training Accuracy = 0.42188
Iter 7680, Minibatch Loss= 387605.156250, Training Accuracy = 0.73438
Iter 8960, Minibatch Loss= 707872.562500, Training Accuracy = 0.60938
Iter 10240, Minibatch Loss= 420275.750000, Training Accuracy = 0.64062
Iter 11520, Minibatch Loss= 547694.500000, Training Accuracy = 0.60938
Iter 12800, Minibatch Loss= 236185.750000, Training Accuracy = 0.75000
Iter 14080, Minibatch Loss= 399002.312500, Training Accuracy = 0.73438
Iter 15360, Minibatch Lo

NameError: name 'saver' is not defined